In [33]:
import json,nltk,re,datetime,collections
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import word_tokenize, pos_tag
from nltk.util import ngrams

def read_words(filename):
    f = open(filename,'r').read().split("\n")
    return dict(zip(f,[1]*len(f)))

if __name__ == "__main__":
    #Start Timer
    start_time = datetime.datetime.now()

    f = open("signal-news1.jsonl","r")
    positive = read_words("opinion-lexicon-English/positive-words.txt")
    negative = read_words("opinion-lexicon-English/negative-words.txt")

    data = []
    pos_story = 0
    neg_story = 0
    pos_words = 0
    neg_words = 0
    temp_set = []
    temp_tokens = []
    lmtzr = WordNetLemmatizer()
    pos_tag_dict = {}

    trigrams = []

    for line in f :
        temp = json.loads(line).get("content")
    
        # Lower the characters
        temp = temp.lower()
    
        #Q1.d remove all URLs
        rule = re.compile(r"http(s)?://[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]")
        temp = rule.sub('', temp)
    
        #Q1.a remove all non-alphacharacters except spaces
        rule = re.compile(r'[^a-z0-9]+')
        temp = rule.sub(' ',temp)
    
        #Q1.b remove all words within 3 characters
        rule = re.compile(r'\W*\b\w{1}\b')
        temp = rule.sub('', temp)
    
        #Q1.c remove numbers that are fully made of digits
        rule = re.compile(r"([0-9]+ )")
        temp = rule.sub('', temp)
    
    
        #Q2 POS tagger
        temp = nltk.word_tokenize(temp)
        postag = []
        
        for word in temp:
            if word in pos_tag_dict:
                postag.append((word,pos_tag_dict[word]))
            else:
                tagofword = nltk.pos_tag([word])
                tag = nltk.pos_tag([word])[0][1]
                pos_tag_dict[word] = tag
                postag.append((word,pos_tag_dict[word]))
                
              
        #if pos was not setted, then lemmatize may not be useful
        temp = []
        for word, tag in postag:
            if tag.startswith('J'):
                word = lmtzr.lemmatize(word, wordnet.ADJ)
            elif tag == "VBP" or tag == "VBZ":
                word = lmtzr.lemmatize(word, wordnet.ADJ)
            elif tag.startswith('V'):
                word = lmtzr.lemmatize(word, wordnet.VERB)
            elif tag.startswith('N'):
                word = lmtzr.lemmatize(word, wordnet.NOUN)
            elif tag.startswith('R'):
                word = lmtzr.lemmatize(word, wordnet.ADV)
            elif tag.startswith('I'):
                word = lmtzr.lemmatize(word, wordnet.ADJ)
            else :
                word = lmtzr.lemmatize(word, wordnet.NOUN)
            temp.append(word)
            
        
        temp_tokens.extend(temp)
        trigrams.extend(list(ngrams(temp,3)))
        
        pos = 0
        neg = 0
      
        for i in temp:
            if i in positive:
                pos += 1
            elif i in negative:
                neg += 1
      
        if pos > neg :
            pos_story += 1
        elif pos < neg :
            neg_story += 1

        pos_words += pos
        neg_words += neg
        data.append(temp)

    trigrams = collections.Counter(trigrams).most_common(25)
    temp_set = set(temp_tokens)
    end_time = datetime.datetime.now()
    interval = (end_time-start_time).seconds

    print(interval)
    print("Number of tokens:",len(temp_tokens))
    print("Vocabulary size is: ", len(temp_set))
    print(pos_story)
    print(neg_story)
    print(trigrams)
#    print(data[6])

321
Number of tokens: 5762073
Vocabulary size is:  88011
[(('one', 'of', 'the'), 2437), (('on', 'share', 'of'), 2093), (('day', 'move', 'average'), 1979), (('on', 'the', 'stock'), 1567), (('as', 'well', 'as'), 1420), (('in', 'research', 'report'), 1416), (('in', 'research', 'note'), 1375), (('the', 'year', 'old'), 1259), (('the', 'united', 'state'), 1227), (('for', 'the', 'quarter'), 1221), (('average', 'price', 'of'), 1193), (('research', 'report', 'on'), 1177), (('the', 'end', 'of'), 1139), (('research', 'note', 'on'), 1138), (('share', 'of', 'the'), 1133), (('be', 'able', 'to'), 1132), (('in', 'report', 'on'), 1124), (('earnings', 'per', 'share'), 1122), (('buy', 'rating', 'to'), 1075), (('cell', 'phone', 'plan'), 1073), (('phone', 'plan', 'detail'), 1070), (('accord', 'to', 'the'), 1063), (('of', 'the', 'company'), 1057), (('move', 'average', 'price'), 995), (('appear', 'first', 'on'), 992)]
